# LangGraph Utilities Demonstration

This notebook demonstrates the comprehensive utility module for LangGraph agent development, including:

1. 📄 **Prompt Composition** - Merge prompts with examples and placeholders
2. 🧱 **Structured Validation** - Validate agent outputs with Pydantic schemas
3. 🔍 **Dynamic Agent Discovery** - Automatically discover and load agents
4. 🕸️ **Workflow Construction** - Build and chain LangGraph workflows


In [1]:
# Setup and imports
import os
import sys
import json
from pathlib import Path

# Add project root to Python path (go up 2 levels from agents/sandbox/)
project_root = Path('.').resolve().parent.parent
sys.path.insert(0, str(project_root))

# Import our utility module directly from this directory
from langgraph_utils import *

# Setup logging (disable workflow logging for cleaner output)
setup_logging(level="INFO", enable_workflow_logging=False)


## 1. 📄 Prompt Composition Utilities


In [2]:
# 1.1 Basic prompt template management with placeholder discovery and merging
template_text = """
Hello {{name}}, welcome to our {{service}}.

Your request: {{input}}

Examples:
{{examples}}
"""

# Create template instance with required placeholders validation
prompt_template = PromptTemplate(template_text, required_placeholders=['name', 'input'])

# Show discovered placeholders from template parsing
print(f"Discovered placeholders: {prompt_template.discovered_placeholders}")

# Merge template with actual values
merged = prompt_template.merge(
    name="Alice",
    service="Healthcare Navigator",
    input="I need help finding a doctor",
    examples="No examples provided"
)

# Display final merged template
print("Merged prompt:")
print(merged)


Discovered placeholders: ['service', 'input', 'examples', 'name']
Merged prompt:

Hello Alice, welcome to our Healthcare Navigator.

Your request: I need help finding a doctor

Examples:
No examples provided



In [3]:
# 1.2 File-based prompt and examples merging from external files
# Load examples from JSON file and convert to structured data
examples_data = load_examples_file("example_examples.json")
print(f"Loaded {len(examples_data)} examples from JSON")

# Convert examples to markdown format for prompt integration
examples_md = convert_examples_to_markdown(examples_data)
print(f"Converted to markdown ({len(examples_md)} characters)")

# Load base prompt template from markdown file
prompt_content = load_prompt_file("example_prompt.md")

# Merge prompt template with examples and user input
user_input = "I'm looking for a pediatrician in my area who takes Medicaid"
merged_prompt = merge_prompt_with_examples(
    prompt_path="example_prompt.md",
    examples_path="example_examples.json", 
    user_input=user_input
)

# Show final merged prompt ready for LLM
print(f"\nFinal merged prompt ({len(merged_prompt)} characters):")
print(merged_prompt)

Loaded 4 examples from JSON
Converted to markdown (2866 characters)

Final merged prompt (3691 characters):
# Example Agent Prompt Template

You are an intelligent healthcare assistant specializing in analyzing patient requests and providing helpful guidance.

## Your Role
You help patients understand their healthcare options and navigate insurance coverage questions.

## Examples
## Example 1
**Input:**
I need to find a dermatologist who takes my insurance

**Output:**
I can help you find a dermatologist that accepts your insurance. To provide the most accurate recommendations, I'll need to know your insurance plan details and preferred location. Here's what I recommend: 1) Check your insurance provider's website or app for an in-network provider directory, 2) Call your insurance company's customer service line, 3) Contact local dermatology practices directly to verify coverage. Would you like me to help you navigate any of these options?

**Context:**
confidence: 0.9
metadata:
  requ

## 2. 🧱 Structured Output Validation


In [4]:
# 2.1 Structured validation with Pydantic schemas
# Create validators with different modes for schema validation
lenient_validator = create_validator(ExampleAgentOutput, pedantic=False)
pedantic_validator = create_validator(ExampleAgentOutput, pedantic=True)

# Test data - valid structure matching ExampleAgentOutput schema
valid_data = {
    "response": "I can help you find a doctor in your area.",
    "confidence": 0.9,
    "metadata": {"urgency": "medium"}
}

# Validate with lenient mode (allows extra fields)
is_valid, validated, error = lenient_validator.validate(valid_data)
print(f"Valid data validation: {is_valid}")
if validated:
    print(f"Response: {validated.response}")
    print(f"Confidence: {validated.confidence}")

# Test data with validation error (confidence out of bounds)
invalid_data = {
    "response": "I can help you find a doctor.",
    "confidence": 1.5,  # Invalid: exceeds 1.0 limit
    "metadata": {"urgency": "high"}
}

# Show validation failure and error handling
is_valid, validated, error = lenient_validator.validate(invalid_data)
print(f"\nInvalid data validation: {is_valid}")
if error:
    print(f"Validation error: {error}")

# Demonstrate LangChain's direct Pydantic validation approach
direct_valid = ExampleAgentOutput.model_validate(valid_data)
print(f"\nDirect Pydantic validation type: {type(direct_valid)}")
print(f"Direct validation response: {direct_valid.response}")


Valid data validation: True
Response: I can help you find a doctor in your area.
Confidence: 0.9

Invalid data validation: False
Validation error: Validation errors:
  - Field 'confidence': Input should be less than or equal to 1

Direct Pydantic validation type: <class 'langgraph_utils.ExampleAgentOutput'>
Direct validation response: I can help you find a doctor in your area.


## 2.2 LangChain Structured Output with Message Patterns


In [5]:
# 2.2 LangChain structured agent creation with message patterns
# Import LangChain chat models (with fallback to mock mode)
try:
    from langchain_anthropic import ChatAnthropic
    anthropic_llm = ChatAnthropic(model="claude-3-5-haiku-latest", temperature=0)
except ImportError:
    anthropic_llm = None

# Create healthcare agent using prompt + examples + schema
healthcare_agent = create_langchain_structured_agent(
    name="HealthcareMessage",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=ExampleAgentOutput,
    llm=anthropic_llm  # Will use mock mode if None
)

# Test agent with user query - returns structured Pydantic object
result1 = healthcare_agent("Find me an orthopedic surgeon")
print(f"Healthcare agent result type: {type(result1)}")
print(f"Response: {result1.response[:70]}...")
print(f"Confidence: {result1.confidence}")
print(f"Metadata: {list(result1.metadata.keys())}")

# Create reasoning agent with custom system message and different schema
custom_system = "You are an expert healthcare advisor. Provide detailed reasoning for insurance decisions."
reasoning_agent = create_langchain_structured_agent(
    name="ReasoningMessage",
    prompt_path="example_prompt.md", 
    examples_path="example_examples.json",
    output_schema=ChainOfThoughtOutput,
    system_message=custom_system,
    llm=anthropic_llm
)

# Test reasoning agent - returns ChainOfThoughtOutput with structured reasoning
result2 = reasoning_agent("What factors should I consider when choosing insurance?")
print(f"\nReasoning agent result type: {type(result2)}")
print(f"Thinking: {result2.thinking[:60]}...")
print(f"Conclusion: {result2.conclusion[:60]}...")
print(f"Reasoning steps: {len(result2.steps)}")
print(f"Confidence: {result2.confidence}")


2025-06-21 19:15:19,748 - langgraph_utils - INFO - Creating LangChain structured agent: HealthcareMessage
2025-06-21 19:15:19,749 - langgraph_utils - INFO - Creating real LangChain agent with ChatAnthropic
2025-06-21 19:15:26,403 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-21 19:15:26,418 - langgraph_utils - INFO - Agent HealthcareMessage generated structured output: <class 'langgraph_utils.ExampleAgentOutput'>
2025-06-21 19:15:26,418 - langgraph_utils - INFO - Creating LangChain structured agent: ReasoningMessage
2025-06-21 19:15:26,419 - langgraph_utils - INFO - Creating real LangChain agent with ChatAnthropic


Healthcare agent result type: <class 'langgraph_utils.ExampleAgentOutput'>
Response: I can help you find an orthopedic surgeon. To provide the most accurat...
Confidence: 0.85
Metadata: ['request_type', 'urgency', 'requires_followup']


2025-06-21 19:15:30,768 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-21 19:15:30,774 - langgraph_utils - INFO - Agent ReasoningMessage generated structured output: <class 'langgraph_utils.ChainOfThoughtOutput'>



Reasoning agent result type: <class 'langgraph_utils.ChainOfThoughtOutput'>
Thinking: Choosing the right insurance requires a comprehensive evalua...
Conclusion: The ideal insurance plan balances comprehensive coverage, af...
Reasoning steps: 8
Confidence: 0.85


## 3. 🔍 Dynamic Agent Discovery


In [6]:
# 3.1 Dynamic agent discovery from project directory structure
# Create discovery instance pointing to agents directory
discovery = AgentDiscovery(base_path=str(project_root / "agents"))

# Scan filesystem for agent modules and get structured metadata
agents = discovery.discover_agents()
agent_names = discovery.list_agents()
print(f"Discovered {len(agents)} agents: {agent_names[:5]}...")

# Extract detailed agent information (first 3 for demo)
for name in agent_names[:3]:
    info = discovery.get_agent_info(name)
    print(f"\nAgent '{name}':")
    print(f"  Path: {info.path}")
    print(f"  Agent class: {'Found' if info.agent_class else 'None'}")
    print(f"  Factory function: {'Found' if info.factory_function else 'None'}")
    if info.init_error:
        print(f"  Error: {info.init_error[:80]}...")

# Generate full discovery report with all agent metadata
discovery.print_discovery_report()


🔧 Using local database: postgresql://aq_home@[host]


2025-06-21 19:15:34,350 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent workflow_prescription: Error loading module ...workflow_prescription: No module named '.'
2025-06-21 19:15:34,353 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent unit: Error loading module ...prompt_security.tests.unit.test_prompt_security_agent: No module named 'agents.prompt_security_agent'
2025-06-21 19:15:34,355 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent models: Error loading module ...prompt_security.models: No module named '.'
2025-06-21 19:15:34,360 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent models: Error loading module ...chat_communicator.models: No module named '.'
2025-06-21 19:15:34,361 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent regulatory: Error loading module ...regulatory.regulatory: No module named '.'
2025-06-21 19:15:34,400 - langgraph_utils.AgentDiscovery - WARNING - Error loading agent tests: Erro

Discovered 11 agents: ['agents', 'workflow_prescription', 'prompt_security', 'unit', 'models']...

Agent 'agents':
  Path: /Users/aq_home/1Projects/accessa/insurance_navigator/agents/__init__.py
  Agent class: None
  Factory function: None

Agent 'workflow_prescription':
  Path: /Users/aq_home/1Projects/accessa/insurance_navigator/agents/workflow_prescription/__init__.py
  Agent class: None
  Factory function: None
  Error: Error loading module ...workflow_prescription: No module named '.'...

Agent 'prompt_security':
  Path: /Users/aq_home/1Projects/accessa/insurance_navigator/agents/prompt_security/prompt_security.py
  Agent class: Found
  Factory function: Found

🔍 Agent Discovery Report
Base Path: /Users/aq_home/1Projects/accessa/insurance_navigator/agents
Agents Found: 11

📦 agents
   Path: /Users/aq_home/1Projects/accessa/insurance_navigator/agents/__init__.py
   Module: ..
   Description: Agents package for the Insurance Navigator system.
   ⚠️  No instantiation method found

📦 

In [7]:
# 3.2 Agent loading with error handling and introspection
# Attempt to load discovered agents with mock mode fallback
loadable_agents = []
for agent_name in agent_names[:3]:
    try:
        # Load agent instance with mock mode to avoid API dependencies
        agent = discovery.load_agent(agent_name, use_mock=True)
        loadable_agents.append((agent_name, agent))
        print(f"Loaded agent '{agent_name}': {type(agent).__name__}")
    except Exception as e:
        print(f"Failed to load '{agent_name}': {str(e)[:80]}...")

print(f"\nSuccessfully loaded {len(loadable_agents)} agents")

# Introspect loaded agent methods and attributes
if loadable_agents:
    agent_name, agent = loadable_agents[0]
    methods = [method for method in dir(agent) if not method.startswith('_')]
    print(f"{agent_name} available methods: {methods[:10]}...")


Failed to load 'agents': Error instantiating agent 'agents': No instantiation method found for agent 'age...
Failed to load 'workflow_prescription': Agent 'workflow_prescription' has initialization error: Error loading module ......
Failed to load 'prompt_security': Error instantiating agent 'prompt_security': get_config_manager() got an unexpec...

Successfully loaded 0 agents


## 4. 🧠 Quick Agent Prototype


In [8]:
# 4.1 FIXED: Demonstrate LangChain Best Practices in Our Implementation
print("== LangChain Best Practice Integration ===")

# Test the NEW LangChain-compliant function
print("\n🧪 Testing LangChain-Compliant Agent Function")
langchain_agent = create_langchain_structured_agent(
    name="LangChainHelper",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=ExampleAgentOutput,
    llm=None  # Mock mode for demo
)

print(f"✅ Created LangChain agent: {langchain_agent.__name__}")

# Test with direct Pydantic object return (LangChain pattern)
test_query = "Find me a specialist who takes my insurance"
result = langchain_agent(test_query)

print(f"\n🎯 LangChain Pattern Results:")
print(f"   Input: {test_query}")
print(f"   Type returned: {type(result)}")
print(f"   Response: {result.response[:80]}...")
print(f"   Confidence: {result.confidence}")
print(f"   Metadata keys: {list(result.metadata.keys())}")
print(f"   ✅ Direct Pydantic object - no JSON parsing!")

# Compare with our legacy approach
print(f"\n📊 Comparison: Legacy vs LangChain Pattern")

# Legacy approach with validation wrapper
print(f"\n1️⃣ Legacy Pattern (with fixed JSON generation):")
legacy_agent = quick_agent_prototype(
    name="LegacyHelper", 
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=ExampleAgentOutput,
    llm=None
)

legacy_result = legacy_agent(test_query)
print(f"   Returns: Dict with validation wrapper")
print(f"   Validation: {'✅ Passed' if legacy_result['validation_passed'] else '❌ Failed'}")
if legacy_result['validation_passed']:
    print(f"   Structured output: {type(legacy_result['structured_output'])}")
    print(f"   Response: {legacy_result['structured_output'].response[:60]}...")

print(f"\n2️⃣ LangChain Pattern:")
print(f"   Returns: Direct Pydantic {type(result)}")
print(f"   Response: {result.response[:60]}...")
print(f"   ✅ No validation wrapper needed!")

print(f"\n🔑 Key Benefits of LangChain Pattern:")
print("✅ Cleaner API - returns Pydantic objects directly")
print("✅ Better error handling - schema binding at model level")
print("✅ More reliable - uses tool calling instead of prompt requests")
print("✅ Ecosystem integration - works seamlessly with LangChain")
print("✅ Production ready - supports real LLMs with with_structured_output()")

# Demonstrate with Chain-of-Thought schema
print(f"\n🧠 Testing with Chain-of-Thought Schema:")
cot_agent = create_langchain_structured_agent(
    name="ReasoningAgent",
    prompt_path="example_prompt.md", 
    examples_path="example_examples.json",
    output_schema=ChainOfThoughtOutput,
    llm=None
)

cot_result = cot_agent("How should I compare different insurance plans?")
print(f"   Type: {type(cot_result)}")
print(f"   Thinking: {cot_result.thinking[:70]}...")
print(f"   Conclusion: {cot_result.conclusion[:70]}...")
print(f"   Steps: {len(cot_result.steps)} reasoning steps")
print(f"   Confidence: {cot_result.confidence}")

print(f"\n💡 Implementation Note:")
print("Both patterns work and are maintained for compatibility.")
print("Use create_langchain_structured_agent() for new development.")
print("Use quick_agent_prototype() for rapid prototyping with validation details.")


2025-06-21 19:15:34,429 - langgraph_utils - INFO - Creating LangChain structured agent: LangChainHelper
2025-06-21 19:15:34,431 - langgraph_utils - WARNING - No LLM provided, using mock mode for LangChainHelper
2025-06-21 19:15:34,435 - langgraph_utils - INFO - Creating LangChain structured agent: ReasoningAgent
2025-06-21 19:15:34,435 - langgraph_utils - WARNING - No LLM provided, using mock mode for ReasoningAgent


== LangChain Best Practice Integration ===

🧪 Testing LangChain-Compliant Agent Function
✅ Created LangChain agent: LangChainHelper_langchain_agent

🎯 LangChain Pattern Results:
   Input: Find me a specialist who takes my insurance
   Type returned: <class 'langgraph_utils.ExampleAgentOutput'>
   Response: Mock response from LangChainHelper: I can help you with 'Find me a specialist wh...
   Confidence: 0.8
   Metadata keys: ['mock', 'agent', 'input_length', 'timestamp']
   ✅ Direct Pydantic object - no JSON parsing!

📊 Comparison: Legacy vs LangChain Pattern

1️⃣ Legacy Pattern (with fixed JSON generation):
   Returns: Dict with validation wrapper
   Validation: ✅ Passed
   Structured output: <class 'langgraph_utils.ExampleAgentOutput'>
   Response: Mock response from LegacyHelper: I can help you with 'Find m...

2️⃣ LangChain Pattern:
   Returns: Direct Pydantic <class 'langgraph_utils.ExampleAgentOutput'>
   Response: Mock response from LangChainHelper: I can help you with 'Fin...
 

## 4.1 🔧 FIXED: Quick Agent Prototype (JSON Parsing Issue Resolved)

**Problem Identified:** The original `quick_agent_prototype` function was generating plain text mock responses that failed JSON validation, causing the error:
```
Invalid JSON: Expecting value: line 1 column 2 (char 1)
```

**Root Cause:** Mock responses like `"[Mock response for agent..."` start with `[M` which JSON parser interprets as an invalid array.

**Solution Implemented:** 
- Added `_generate_mock_data_for_schema()` function that creates valid JSON matching the Pydantic schema
- Updated `quick_agent_prototype()` to generate structured JSON when a schema is provided
- Maintained backward compatibility for unstructured outputs

Let's demonstrate the fix:


In [9]:
# 4.2 FIXED Quick Agent Prototype Demo - Addresses JSON Parsing Issues
print("=== 🔧 FIXED Quick Agent Prototype Demo ===")
print("This demonstrates the fix for the JSON parsing error identified in the RCA.")

# Test 1: Create agent with structured output (ExampleAgentOutput schema)
print("\n🧪 Test 1: HealthcareHelper with ExampleAgentOutput Schema")
fixed_prototype_agent = quick_agent_prototype(
    name="HealthcareHelper",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=ExampleAgentOutput,
    pedantic_validation=False,
    llm=None  # Mock mode now generates valid JSON!
)

print(f"✅ Created agent: {fixed_prototype_agent.__name__}")
print(f"   Documentation: {fixed_prototype_agent.__doc__}")

# Test the fixed prototype
test_inputs = [
    "I need to find a dermatologist", 
    "Does my insurance cover MRI scans?",
    "I'm having severe headaches and need urgent care"
]

for i, test_input in enumerate(test_inputs, 1):
    print(f"\n🔬 Test {i}: {test_input}")
    result = fixed_prototype_agent(test_input)
    
    print(f"   Agent: {result['agent_name']}")
    print(f"   Validation: {'✅ PASSED' if result['validation_passed'] else '❌ FAILED'}")
    
    if result.get('validation_error'):
        print(f"   ❌ Error: {result['validation_error']}")
    else:
        # Show successful structured output
        structured = result['structured_output']
        print(f"   ✅ Response: {structured.response[:100]}...")
        print(f"   ✅ Confidence: {structured.confidence}")
        print(f"   ✅ Metadata: {structured.metadata}")
        
        # Show the valid JSON that was generated
        print(f"   📋 Valid JSON Generated: {result['raw_output'][:200]}...")

print(f"\n🧠 Test 2: ChainOfThought Schema Demonstration")
# Test with a different schema to show versatility
thinking_agent = quick_agent_prototype(
    name="ThinkingAgent",
    prompt_path="example_prompt.md", 
    examples_path="example_examples.json",
    output_schema=ChainOfThoughtOutput,
    pedantic_validation=False,
    llm=None
)

thinking_result = thinking_agent("How should I choose between different health insurance plans?")
print(f"🔬 Chain-of-Thought Test:")
print(f"   Validation: {'✅ PASSED' if thinking_result['validation_passed'] else '❌ FAILED'}")

if thinking_result['validation_passed']:
    cot = thinking_result['structured_output']
    print(f"   🤔 Thinking: {cot.thinking[:80]}...")
    print(f"   💡 Conclusion: {cot.conclusion[:80]}...")
    print(f"   📝 Steps: {len(cot.steps)} reasoning steps")
    print(f"   🎯 Confidence: {cot.confidence}")

print(f"\n🆚 Test 3: Comparison - Agent WITHOUT Schema (Plain Text)")
# Show what happens without a schema (plain text mock)
unstructured_agent = quick_agent_prototype(
    name="PlainTextAgent",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json", 
    output_schema=None,  # No schema = plain text mock
    llm=None
)

plain_result = unstructured_agent("I need help with insurance")
print(f"📄 Plain Text Agent Result:")
print(f"   Output: {plain_result['raw_output']}")
print(f"   Validation: {'N/A (no schema)' if plain_result.get('structured_output') is None else 'Has validation'}")

print(f"\n✅ Summary: JSON Parsing Issue RESOLVED!")
print("✅ Mock responses now generate valid JSON when schema is provided")
print("✅ Validation passes successfully for all structured outputs")
print("✅ Multiple schemas supported (ExampleAgentOutput, ChainOfThoughtOutput)")
print("✅ Backward compatibility maintained for unstructured outputs")


=== 🔧 FIXED Quick Agent Prototype Demo ===
This demonstrates the fix for the JSON parsing error identified in the RCA.

🧪 Test 1: HealthcareHelper with ExampleAgentOutput Schema
✅ Created agent: HealthcareHelper_agent
   Documentation: Auto-generated agent function for 'HealthcareHelper' with structured output

🔬 Test 1: I need to find a dermatologist
   Agent: HealthcareHelper
   Validation: ✅ PASSED
   ✅ Response: Mock response from HealthcareHelper: I can help you with 'I need to find a dermatologist...'. This i...
   ✅ Confidence: 0.8
   ✅ Metadata: {'mock': True, 'agent': 'HealthcareHelper', 'input_length': 30, 'timestamp': 'mock_timestamp'}
   📋 Valid JSON Generated: {
  "response": "Mock response from HealthcareHelper: I can help you with 'I need to find a dermatologist...'. This is a simulated response for testing purposes.",
  "confidence": 0.8,
  "metadata": {...

🔬 Test 2: Does my insurance cover MRI scans?
   Agent: HealthcareHelper
   Validation: ✅ PASSED
   ✅ Response: Mo

In [10]:
# 4.1 Create a quick prototype agent
print("=== Quick Agent Prototype Demo ===")

# Create a prototype agent with structured output
prototype_agent = quick_agent_prototype(
    name="HealthcareHelper",
    prompt_path="example_prompt.md",
    examples_path="example_examples.json",
    output_schema=ExampleAgentOutput,
    pedantic_validation=False,
    llm=None  # Using mock mode (no actual LLM)
)

print(f"✅ Created prototype agent: {prototype_agent.__name__}")

# Test the prototype with different inputs
test_inputs = [
    "I need to find a dermatologist",
    "Does my insurance cover MRI scans?", 
    "I'm having severe headaches and dizziness"
]

print(f"\n=== Testing Prototype Agent ===")
for i, test_input in enumerate(test_inputs):
    print(f"\n🧪 Test {i+1}: {test_input}")
    result = prototype_agent(test_input)
    
    print(f"   Agent: {result['agent_name']}")
    print(f"   Output: {result['raw_output'][:100]}...")
    print(f"   Validation: {'✅ Passed' if result['validation_passed'] else '❌ Failed'}")
    
    if result.get('validation_error'):
        print(f"   Error: {result['validation_error'][:100]}...")
    
    if result.get('structured_output'):
        print(f"   Confidence: {result['structured_output'].confidence}")


=== Quick Agent Prototype Demo ===
✅ Created prototype agent: HealthcareHelper_agent

=== Testing Prototype Agent ===

🧪 Test 1: I need to find a dermatologist
   Agent: HealthcareHelper
   Output: {
  "response": "Mock response from HealthcareHelper: I can help you with 'I need to find a dermatol...
   Validation: ✅ Passed
   Confidence: 0.8

🧪 Test 2: Does my insurance cover MRI scans?
   Agent: HealthcareHelper
   Output: {
  "response": "Mock response from HealthcareHelper: I can help you with 'Does my insurance cover M...
   Validation: ✅ Passed
   Confidence: 0.8

🧪 Test 3: I'm having severe headaches and dizziness
   Agent: HealthcareHelper
   Output: {
  "response": "Mock response from HealthcareHelper: I can help you with 'I'm having severe headach...
   Validation: ✅ Passed
   Confidence: 0.8


## 5. 🕸️ LangGraph Workflow Construction


In [11]:
# 5.1 Proper LangGraph workflow construction following official patterns
# Import LangGraph components and define state schema
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """Proper LangGraph state schema following official documentation"""
    messages: Annotated[list[BaseMessage], add_messages]
    user_input: str
    analysis: str
    urgency_level: str
    keywords: list
    processing_result: str
    final_response: str

# Define node functions that work with dictionary state (LangGraph standard)
def analyze_input(state: GraphState) -> GraphState:
    """Analyze user input and extract key information"""
    user_input = state.get("user_input", "")
    analysis = f"Analyzed: {user_input[:50]}..."
    
    return {
        "analysis": analysis,
        "messages": [{"role": "assistant", "content": f"Analysis complete for: {user_input[:30]}..."}]
    }

def process_request(state: GraphState) -> GraphState:
    """Process the analyzed request"""
    analysis = state.get("analysis", "No analysis")
    processing_result = f"Processed based on: {analysis}"
    
    return {
        "processing_result": processing_result,
        "messages": [{"role": "assistant", "content": "Request processed successfully"}]
    }

def format_response(state: GraphState) -> GraphState:
    """Format the final response for output"""
    processing_result = state.get("processing_result", "No result")
    final_response = f"Final response: {processing_result}"
    
    return {
        "final_response": final_response,
        "messages": [{"role": "assistant", "content": final_response}]
    }

# Conditional routing function for LangGraph
def should_process(state: GraphState) -> str:
    """Determine workflow routing based on analysis results"""
    return "process" if state.get("analysis") else "end"

print("LangGraph functions defined with proper state schema")


LangGraph functions defined with proper state schema


In [12]:
# 5.2 Build and test healthcare workflow using proper LangGraph patterns
# Create LangGraph StateGraph with proper state schema
workflow = StateGraph(GraphState)

# Add nodes to the graph using official LangGraph pattern
workflow.add_node("analyze", analyze_input)
workflow.add_node("process", process_request) 
workflow.add_node("format", format_response)

# Define workflow routing with conditional edges
workflow.set_entry_point("analyze")
workflow.add_conditional_edges(
    "analyze",
    should_process,
    {"process": "process", "end": "format"}
)
workflow.add_edge("process", "format")

# Compile the workflow (LangGraph requirement)
compiled_workflow = workflow.compile()
print("Workflow compiled successfully")

# Test workflow execution with proper state dictionary
initial_state = {
    "user_input": "I need help finding a cardiologist",
    "messages": [],
    "analysis": "",
    "urgency_level": "",
    "keywords": [],
    "processing_result": "",
    "final_response": ""
}

try:
    # Execute workflow and capture results (proper LangGraph invoke)
    result = compiled_workflow.invoke(initial_state)
    print(f"Workflow completed")
    print(f"Messages: {len(result.get('messages', []))}")
    print(f"Analysis: {result.get('analysis', 'None')}")
    print(f"Final response: {result.get('final_response', 'No response')[:100]}...")
except Exception as e:
    print(f"Workflow execution failed: {str(e)}")


Workflow compiled successfully
Workflow completed
Messages: 3
Analysis: Analyzed: I need help finding a cardiologist...
Final response: Final response: Processed based on: Analyzed: I need help finding a cardiologist......


## 6. 🔁 Workflow Chaining


In [13]:
# 6.1 Parallel node execution and workflow chaining with proper LangGraph patterns
# Define parallel processing nodes for simultaneous execution
def analyze_urgency(state: GraphState) -> GraphState:
    """Analyze urgency level in parallel with main processing"""
    user_input = state.get("user_input", "")
    urgency = "high" if any(word in user_input.lower() for word in ["urgent", "emergency", "pain"]) else "normal"
    
    return {
        "urgency_level": urgency,
        "messages": [{"role": "assistant", "content": f"Urgency assessed: {urgency}"}]
    }

def extract_keywords(state: GraphState) -> GraphState:
    """Extract keywords in parallel with main processing"""
    user_input = state.get("user_input", "")
    keywords = [word for word in user_input.lower().split() if len(word) > 4]
    
    return {
        "keywords": keywords,
        "messages": [{"role": "assistant", "content": f"Keywords extracted: {', '.join(keywords[:3])}"}]
    }

# Create enhanced workflow with parallel nodes
enhanced_workflow = StateGraph(GraphState)

# Add parallel processing nodes that can run simultaneously
enhanced_workflow.add_node("analyze_input", analyze_input)
enhanced_workflow.add_node("analyze_urgency", analyze_urgency)  # Parallel node
enhanced_workflow.add_node("extract_keywords", extract_keywords)  # Parallel node
enhanced_workflow.add_node("process_request", process_request)
enhanced_workflow.add_node("format_response", format_response)

# Set up parallel execution pattern from LangGraph documentation
enhanced_workflow.set_entry_point("analyze_input")

# From analyze_input, branch to two parallel nodes
enhanced_workflow.add_edge("analyze_input", "analyze_urgency")
enhanced_workflow.add_edge("analyze_input", "extract_keywords")

# Both parallel nodes feed into process_request
enhanced_workflow.add_edge("analyze_urgency", "process_request")
enhanced_workflow.add_edge("extract_keywords", "process_request")

# Final processing
enhanced_workflow.add_edge("process_request", "format_response")

# Compile the enhanced workflow
compiled_enhanced = enhanced_workflow.compile()
print("Enhanced workflow with parallel nodes compiled")

# Test parallel execution
parallel_state = {
    "user_input": "I need urgent help finding a cardiologist for chest pain",
    "messages": [],
    "analysis": "",
    "urgency_level": "",
    "keywords": [],
    "processing_result": "",
    "final_response": ""
}

try:
    # Execute workflow with parallel node processing
    parallel_result = compiled_enhanced.invoke(parallel_state)
    print(f"Parallel workflow completed")
    print(f"Analysis: {parallel_result.get('analysis', 'None')}")
    print(f"Urgency: {parallel_result.get('urgency_level', 'None')}")
    print(f"Keywords: {parallel_result.get('keywords', [])}")
    print(f"Final response: {parallel_result.get('final_response', 'No response')[:100]}...")
    
except Exception as e:
    print(f"Parallel workflow failed: {str(e)}")


Enhanced workflow with parallel nodes compiled
Parallel workflow completed
Analysis: Analyzed: I need urgent help finding a cardiologist for ches...
Urgency: high
Keywords: ['urgent', 'finding', 'cardiologist', 'chest']
Final response: Final response: Processed based on: Analyzed: I need urgent help finding a cardiologist for ches......


## 7. 📋 Summary

### Core LangGraph Components Demonstrated

1. **📄 Prompt Composition**: Template merging, example conversion, placeholder substitution
2. **🧱 Structured Validation**: Pydantic schema validation, error handling, LangChain integration
3. **🔍 Dynamic Agent Discovery**: Filesystem scanning, module introspection, agent loading
4. **🧠 Quick Prototyping**: Combined prompt+validation, structured output generation
5. **🕸️ LangGraph Workflows**: StateGraph construction, node/edge definition, proper state management
6. **⚡ Parallel Execution**: Simultaneous node processing, following LangGraph parallelization patterns

### Key Data Flow Patterns

- **Files → Templates → Structured Output**: `prompt.md` + `examples.json` → `PromptTemplate` → `Pydantic Object`
- **Agent Discovery → Loading**: `filesystem scan` → `AgentInfo` → `loaded agent instance`
- **LangGraph Execution**: `GraphState dict` → `node functions` → `updated state dict`
- **Parallel Processing**: `initial_state` → `[node_1, node_2]` → `merged_state` → `next_node`

### Core LangGraph Usage Patterns

- `StateGraph(GraphState)` - Official LangGraph state management with TypedDict schema
- `workflow.add_node(name, function)` - Node registration with dictionary state functions
- `workflow.add_conditional_edges()` - Conditional routing based on state values
- `workflow.compile().invoke(state_dict)` - Proper workflow compilation and execution
- **Parallel Nodes**: Multiple nodes receiving same input, outputs merged automatically


## 7. 📋 Summary

This notebook demonstrated comprehensive LangGraph workflow patterns and utilities:

### ✅ What We Accomplished

1. **📄 Prompt Composition**: 
   - Loaded prompts from files with placeholder merging
   - Converted JSON examples to Markdown format automatically
   - Merged templates with user input and examples

2. **🧱 Structured Validation**:
   - Created validators for Pydantic schemas
   - Demonstrated lenient vs pedantic validation modes
   - Showed helpful error formatting

3. **🔍 Dynamic Agent Discovery**:
   - Automatically discovered agents in the project directory
   - Loaded agent information without hardcoded paths
   - Attempted agent instantiation with error handling

4. **🧠 Quick Agent Prototyping**:
   - Combined prompt merging with structured validation
   - Created prototype agents with minimal code
   - Tested with multiple inputs

5. **🕸️ LangGraph Workflow Construction**:
   - Built workflows using proper StateGraph patterns
   - Added nodes, edges, and conditional routing
   - Demonstrated proper state management with TypedDict

6. **⚡ Parallel Node Execution**:
   - Created workflows with simultaneous node processing
   - Implemented LangGraph parallelization patterns
   - Showed automatic state merging from parallel branches

### 🚀 Next Steps

- Integrate with your existing agents using the discovery system
- Create custom GraphState schemas for your specific use cases
- Build complex multi-agent workflows with parallel processing
- Add callback handlers for token usage tracking (see [LangChain discussion](https://github.com/langchain-ai/langchain/discussions/24683))

### 📖 Usage Tips

- Use proper TypedDict state schemas for LangGraph workflows
- Follow official LangGraph patterns for state management
- Create pedantic validators for production systems
- Use the discovery system to avoid hardcoded agent imports
- Implement parallel nodes for performance optimization
